In [ ]:
#hide
from nbdev.showdoc import *
# default_exp animation_helper
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Google Colab preparations

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

IN_MAIN = __name__ == '__main__'

In [ ]:
#Infrastructure for copying notebooks
if IN_COLAB and IN_MAIN:
    home_dir = '/content/drive/MyDrive/Colab Notebooks/Ecosystems/v3'
if IN_COLAB and IN_MAIN:
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append(home_dir)
    %cd $home_dir
    

In [ ]:
if IN_COLAB and IN_MAIN:
    !cd;pip -q install import-ipynb
    !cd;pip -q install stable-baselines3[extra]
    !cd;apt install swig
    !cd;pip -q install box2d box2d-kengz
    #verbose = 0

# Preliminary imports

In [ ]:
#export
# import json
# import random

# from IPython.display import HTML
# import gym
# from gym import spaces
# # This has to be imported before our own notebook imports.
# import import_ipynb
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
# import pandas as pd
import seaborn as sns

# from animal_classes import Ecosystem, MultiSheep, SimpleSheep, Terrain
# import ecoenv
# from ecoenv import EcoEnv
# from perception import Perception
# from stable_baselines3 import PPO, A2C, SAC, DDPG, TD3 # , DQN
# from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
# from stable_baselines3.common.evaluation import evaluate_policy
# from utility import distance, draw_objects, motion_diagram, normalize

In [ ]:
#export animation_helper
class AnimationHelper:
    def __init__(self, env, model):
        self.env = env
        self.model = model

    def init_animation(self, show_consumed=True):

        env = self.env
        self.show_consumed = show_consumed

        # create a figure and axes
        # self.fig = plt.figure(figsize=(12, 12))
        # self.ax1 = plt.subplot(1, 1, 1)
        # self.ax1 = plt.axes()
        self.fig, self.ax1 = plt.subplots()
        self.fig.set_size_inches(12,12)

        # set up the subplots as needed
        s = env.ecosystem.terrain.space
        lower_left = [s[0, 0], s[1, 0]]
        x_side = s[0, 1] - s[0, 0]
        y_side = s[1, 1] - s[1, 0]
        x_lim, y_lim = s[0], s[1]

        self.ax1.set_xlim(x_lim)
        self.ax1.set_ylim(y_lim)

        self.obs = self.env.reset()

        # TODO: Add all circles as is done with objects below.
        self.circle = matplotlib.patches.Circle(
            env.position, next(iter(env.agent.distances.values())), fc="y", alpha=0.1
        )
        self.reward_radius = env.agent.hyperparameters["close"]
        self.reward_circle = matplotlib.patches.Circle(
            env.position, self.reward_radius, fc="r", alpha=0.3
        )
        self.ax1.add_patch(self.circle)
        self.ax1.add_patch(self.reward_circle)

        # This region might not be needed if we only plotting region is the terrain
        # space.
        env_region = matplotlib.patches.Rectangle(
            lower_left, x_side, y_side, fc="b", alpha=0.05
        )
        self.ax1.add_patch(env_region)

        # objects_plot = ax1.scatter(objects[:,0], objects[:,1], marker='x')
        self.objects_plot = {}
        for name, pts in env.ecosystem.terrain.objects.items():
            self.objects_plot[name] = self.ax1.scatter(pts[:, 0], pts[:, 1], label=name)
        self.consumed_plot = {
            name: self.ax1.scatter([], [], marker="x", label=f"Consumed {name}")
            for name in env.ecosystem.terrain.objects.keys()
        } if self.show_consumed else {}

        plt.legend()

        # create objects that will change in the animation. These are
        # initially empty, and will be given new values for each frame
        # in the animation.
        self.txt_title = self.ax1.set_title("Initial configuration")
        (self.line1,) = self.ax1.plot(
            [], [], "g", lw=2
        )  # ax.plot returns a list of 2D line objects
        (self.line2,) = self.ax1.plot([], [], "r", lw=2)
        self.trace = []

        # obs_action stores observation, action tuples.
        self.obs_action = []

        return (
            self.line1,
            *self.objects_plot.values(),
            *self.consumed_plot.values(),
            self.circle,
            self.reward_circle,
        )

    ### drawframe cell starts here
    # animation function. This is called sequentially
    def _gen_title_string(self, n):
        title = [f'Frame: {n:4d}']
        for name, v in self.env.agent.interoception.items():
            title.append(f'{name}: {v:.2f}')

        terrain = self.env.ecosystem.terrain
        title.append(f'Happiness: {self.env.agent.happiness(terrain):.2f}')
        return ' '.join(title)

    def drawframe(self, n):
        obs = self.obs
        model = self.model
        env = self.env

        action, _states = model.predict(obs, deterministic=True)
        self.obs_action.append((obs, action))
        self.obs, reward, dones, info = env.step(action)
        self.trace.append(env.position.copy())
        x, y = zip(*self.trace)
        self.line1.set_data(x, y)
        for name, ax_obj in self.objects_plot.items():
            ax_obj.set_offsets(env.ecosystem.terrain.objects[name])

        if self.show_consumed:
            for name, ax_obj in self.consumed_plot.items():
                ax_obj.set_offsets(
                    np.array(env.ecosystem.terrain.consumed[name]).reshape(-1, 2)
                )

        self.circle.center = env.position
        self.reward_circle.center = env.position

        # title = f'Frame = {n:4d}, '\
        #     f'Energy: {env.agent.interoception["energy"]:.2f}, '\
        #     f'Water: {env.agent.interoception["water"]:.2f}, '\
        #     f'Protein: {env.agent.interoception["protein"]:.2f}, '\
        #     f'Happiness: {env.agent.happiness():.2f}'
        # self.txt_title.set_text(title)
        self.txt_title.set_text(self._gen_title_string(n))

        if dones:
            print(f'Done @{n}, {env.agent.age}, {env.ecosystem.terrain.counter}')
            self.trace.clear()
            env.reset()
        return (
            self.line1,
            *self.objects_plot.values(),
            *self.consumed_plot.values(),
            self.circle,
            self.reward_circle,
        )


if False and __name__ == "__main__":
    env = MothEnv()
    a = AnimationHelper(env, 4)
    print("Some elementary test should be here")



In [ ]:
# a = AnimationHelper(env, 4)

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted animal_classes.ipynb.
Converted animation_helper.ipynb.
Converted ecoenv.ipynb.
Converted happiness.ipynb.
Converted index.ipynb.
Converted perception.ipynb.
Converted reflex_agent.ipynb.
Converted tyckande.ipynb.
Converted utility.ipynb.
Converted worlds.ipynb.
